In [ ]:
import random
import util
from test_framework.key import generate_key_pair, ECKey, ECPubKey, jacobi_symbol, SECP256K1_FIELD_SIZE, SECP256K1_ORDER, tagged_hash
from test_framework.messages import sha256

# 1.1 Introduction to Schnorr Signatures

* Part 1: Schnorr Signatures.
* Part 2: Deterministic Nonces.

## Part 1: Schnorr Signatures

[bip-schnorr](https://github.com/bitcoinops/bips/blob/V0.1/bip-schnorr.mediawiki) defines a signature/verifier scheme, as well as pubkey and signature encodings.

## 32 Byte Public Keys

A secp256k1 curve point consists of an X coordinate (32 bytes) and a Y coordinate (32 bytes). To save 32 bytes in the encoding of a point, we can encode only the X coordinate.

For a given X coordinate on the secp256k1 curve, there are two possible curve points:

* `y^2 = x^3 + 7` (Two Y coordinate values for a given X coordinate)
    * For `x`, both `(x, y)` and `(x, -y)` are valid curve points (where `-y` is `SECP256K1_FIELD_SIZE - y` since all arithmetic involving coordinates is modulo `SECP256K1_FIELD_SIZE`).
    * One of the Y coordinates is even, and the other is odd (since `SECP256K1_FIELD_SIZE` is odd).
    * One of the Y coordinates is a quadratic residue (has a square root modulo the field size), and the other is not.

If we constrain a private key `d` such that the Y coordinate of `P = dG` is a quadratic residue modulo `SECP256K1_FIELD_SIZE`, then it's possible to unambigiously determine the Y coordinate from the X coordinate.

* private keys `d` and `SECP256K1_ORDER - d` give points `P = (x, y)` and `P' = (x, -y)` respectively.
* Only one will have a Y coordinate which is a quadratic residue modulo the field size. If a randomly generated private key `d` does not yield a valid public key `P`, then we can negate `d` to obtain a valid private key.

Whether a scalar is a quadratic residue modulo the secp256k1 field size can be determined by its [jacobi symbol](http://en.wikipedia.org/wiki/Jacobi_symbol). The Test Framework provides a `jacobi_symbol()` function. If `jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1`, then `y` is a quadratic residue.

## Signing and verifying

The schnorr signature equation is the following:

* `S = R + H(x(R)|P|m) * P`

Signing involves generating a secret nonce first.

* Generate secret scalar `k`

Then computing `s` from:

* `s = k + H(x(R)|P|m) * d`

The resulting signature is:

* `x(R), s`

![test](images/schnorr0.jpg)

### 32 Byte R values

In exactly the same way we encoded public keys to be 32 bytes by constraining them to have a Y coordinate that is a quadratic residue, we constrain the nonce such that the nonce point has a Y coordinate that is a quadratic residue.

If a randomly generated nonce value `k` does not yield a point `R` which has a Y coordinate which is a quadratic residue, we use `-k` to get the inverse nonce point that does have a Y coordinate which is a quadratic residue.

#### 1.1.1 Example: Calculating a valid nonce

In [ ]:
# Generate a random value and its assoctiated curve point. We can use the generate_key_pair convenience function.
k, R = generate_key_pair()

# Find y and -y
y = R.get_y()
minus_y = SECP256K1_FIELD_SIZE - y
print("y = {}".format(y))
print("-y = {}\n".format(minus_y))

# One of y and -y will be even and the other will be odd
print("y is {}".format("odd" if y % 2 else "even"))
print("-y is {}\n".format("odd" if minus_y % 2 else "even"))

# One of y and -y will be a quadratic residue and the other will not
print("y is {}a quadratic residue".format("" if jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1 else "not "))
print("-y is {}a quadratic residue\n".format("" if jacobi_symbol(minus_y, SECP256K1_FIELD_SIZE) == 1 else "not "))

print("k is {}a valid nonce".format("" if jacobi_symbol(y, SECP256K1_FIELD_SIZE) == 1 else "not "))
print("-k is {}a valid nonce".format("" if jacobi_symbol(minus_y, SECP256K1_FIELD_SIZE) == 1 else "not "))

#### 1.1.2 _Programming Exercise:_ Verify that inverse nonce values `k` and `-k` generate inverse points `R` and `-R`

In [ ]:
# Generate a random scalar and its associated curve point
k, R =  # TODO: implement

# Find the x- and y-coordinates from R
# Use the get_x() and get_y() methods
R_x =  # TODO: implement
R_y =  # TODO: implement

print("R_x: {}".format(R_x))
print("R_y: {}\n".format(R_y))

# Find k's inverse (SECP256K1_ORDER - k)
# Extract the secret value from k using .secret
minus_k =  # TODO: implement

# Generate the key pair from minus_k using generate_key_pair() function with minus_k as an argument
minus_k_key, minus_R =  # TODO: implement

# Find the x- and y-coordinates from -R
minus_R_x =  # TODO: implement
minus_R_y =  # TODO: implement

print("minus_R_x: {}".format(minus_R_x))
print("minus_R_y: {}\n".format(minus_R_y))

assert R_x == minus_R_x
assert SECP256K1_FIELD_SIZE - R_y == minus_R_y

print("Success!")

#### 1.1.3 _Programming Exercise:_ Sign a message with Schnorr

* Sign the message with the provided key pair below.

In [ ]:
msg = sha256(b'message')

# Generate a private/public key pair
d, P = generate_key_pair()

# Generate a nonce scalar and associated nonce point
k, R = generate_key_pair()

# Check that nonce is quadratic residue modulo the field size
# If not, negate k
# Method: jacobi_symbol(R.get_y(), SECP256K1_FIELD_SIZE)
if  # TODO: implement
    k.negate()
# Note that there is no need to negate R, since we only use the x value of R below

# The same thing for the public key
if # TODO: implement
    d.negate()


# Generate s = k + sha256(R_x|P_x|msg) * d
# Method: tagged_hash("Tag", bytes) will give you the byte digest of the tagged SHA256 of some bytes
# use `BIPSchnorr` as the tag, as specified by the BIP
# Turn that digest into a ECKey object called h, and then set s = k + h * d
R_x_bytes = R.get_xonly_bytes()
P_x_bytes = P.get_xonly_bytes()
h_bytes =  # TODO: implement
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig =  # TODO: implement

print("Signature: {}\n".format(sig.hex()))

# Verify the signature
assert P.verify_schnorr(sig, msg)

print("Success!")

## Part 2: Deterministic Nonces for schnorr signatures

So far we have used a random secret nonce for creating schnorr signatures. This has the disadvantage that the the user must rely on the robustness of the random generator for each signing rounds. If the nonce generator is compromised or even biased, the private key can be derived for a given signature and known nonce.

For **single signer schnorr signatures**, bip-schnorr proposes a deterministic nonce generation scheme.

* `k = sha256(d|msg)`

#### 1.1.4 _Programming Exercise:_ Signing schnorr with deterministic nonce

* Create a Schnorr signature with bip-schnorr's deterministic nonce scheme
* Compare this signature to the private key class method `ECKey.sign_schnorr(msg)`

In [ ]:
msg = sha256(b'message')
d, P = generate_key_pair()
print("message = {}".format(msg.hex()))
print("pubkey = {}\n".format(P))

# Check that the pubkey is quadratic residue modulo the field size
# If not, negate d
if # TODO: implement
    d.negate()

# Generate the nonce value k deterministically and get the nonce point R
# Method: use tagged_hash("BIPSchnorrDerive", bytes) on the private key and message
k_bytes =  # TODO: implement
k, R =  # TODO: implement

# Check that nonce is quadratic residue modulo the field size
# If not, negate k
# Method: jacobi_symbol(int(y(R)), SECP256K1_FIELD_SIZE)
if  # TODO: implement
    k.negate()

print("nonce: {}".format(k))
print("nonce point: {}\n".format(R))

# Generate s = k + sha256(R_x|P_x|msg) * d
# Method: tagged_hash("Tag", bytes) will give you the byte digest of the tagged SHA256 of some bytes
# use `BIPSchnorr` as the tag, as specified by the BIP
# Turn that digest into a ECKey object called h, and then set s = k + h * d
R_x_bytes = R.get_xonly_bytes()
P_x_bytes = P.get_xonly_bytes()
h_bytes =  # TODO: implement
h = ECKey().set(h_bytes)
s = k + h * d

print("R: {}".format(R))
print("s: {}\n".format(s.get_bytes().hex()))

# Generate sig = R_x|s
# Method: get the x bytes from R and concatenate with the secret bytes from s
sig =  # TODO: implement

print("Signature: {}\n".format(sig.hex()))

# Generate a signature using the ECKey.sign_schnorr(msg) method
# This generates the nonce deterministically, so should return the same signature
sig2 = d.sign_schnorr(msg)

# Verify and compare signature(s)
assert P.verify_schnorr(sig, msg)
assert P.verify_schnorr(sig2, msg)
assert sig == sig2

print("Success!")

**Congratulations!** In this chapter, you have:

- Used jacobi_symbol() to determine if a random scalar is a valid bip-schnorr nonce
- Created and verified a valid schnorr signature for a public key P and message m
- Generated a deterministic nonce using a hash digest of the public key and message